# Mock data
This notebook prepares some mock data to be used in the GUI demonstrator>

In [3]:
from aurora.schemas.data_schemas import BatterySpecs, BatteryComposition, BatteryCapacity, BatteryMetadata, BatterySample
import datetime
import json, random
import pandas as pd
import numpy as np
import names

In [4]:
STD_SPECS = pd.read_csv('sample_specs.csv', dtype=str)
STD_SPECS

,manufacturer,composition.description,capacity.nominal,capacity.units,form_factor
0,BIG-MAP,C | LP57 | NMC811,1.539,mAh,2032
1,BIG-MAP,C | LP57 | LNO,1.539,mAh,2032
2,BIG-MAP,Si-C | LP57 | NMC811,1.539,mAh,2032
3,BIG-MAP,Si-C | LNMO | NMC622,1.539,mAh,2032
4,BIG-MAP,C | LP57 | NMC811,4.618,mAh,2032
5,BIG-MAP,C | LP57 | LNO,4.618,mAh,2032
6,BIG-MAP,Si-C | LP57 | NMC811,4.618,mAh,2032
7,BIG-MAP,Si-C | LNMO | NMC622,4.618,mAh,2032
8,Conrad energy,Li-based,45,mAh,2032
9,Conrad energy,Li-based,60,mAh,2032


In [5]:
STD_SPECS.keys()

Index(['manufacturer', 'composition.description', 'capacity.nominal',
       'capacity.units', 'form_factor'],
      dtype='object')

In [6]:
for key in STD_SPECS.keys():
    print(f'{key:14}:  {STD_SPECS.get(key).unique().tolist()}')

manufacturer  :  ['BIG-MAP', 'Conrad energy', 'Maxell']
composition.description:  ['C | LP57 | NMC811', 'C | LP57 | LNO', 'Si-C | LP57 | NMC811', 'Si-C | LNMO | NMC622', 'Li-based', 'LiMgO2-based']
capacity.nominal:  ['1.539', '4.618', '45', '60']
capacity.units:  ['mAh']
form_factor   :  ['2032', '2016']


In [7]:
def random_datetime(
        start=datetime.datetime(2020, 1, 1, 0, 0, 0, tzinfo=datetime.timezone.utc),
        end=datetime.datetime.now(tz=datetime.timezone.utc)):
    """Generate a random datetime between `start` and `end`"""
    return start + datetime.timedelta(
        # Get a random amount of seconds between `start` and `end`
        seconds=random.randint(0, int((end - start).total_seconds())))

In [8]:
def create_random_sample():
    # pick one of the available specs
    random_spec = STD_SPECS.sample().iloc[0]
    battery_id = random.randint(0, 1000)
    sample = BatterySample(
        manufacturer = random_spec.manufacturer,
        composition = BatteryComposition(description=random_spec.get('composition.description')),
        form_factor = random_spec.form_factor,
        capacity = BatteryCapacity(nominal=float(random_spec.get('capacity.nominal')), units=random_spec.get('capacity.units')),
        metadata = BatteryMetadata(
            name = f'{names.get_first_name()}_{battery_id}',
            creation_datetime=random_datetime(),
            creation_process='Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed mollis arcu ac risus vestibulum, ac commodo nunc euismod. Vivamus suscipit eros nec justo faucibus, et eleifend diam eleifend.'
        ),
        battery_id = battery_id,
    )
    return sample

Now we generate 20 random samples and save the to a JSON file.

In [9]:
samples = [create_random_sample() for _ in range(40)]
samples = sorted(samples, key=lambda s: s.battery_id)

In [10]:
class BatterySampleEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, datetime.datetime):
                return obj.isoformat()
            else:
                super().default(obj)

json.dump([s.dict() for s in samples], open('available_samples.json', 'w'), cls=BatterySampleEncoder)
json.dump({s.battery_id: s.dict() for s in samples}, open('available_samples_id.json', 'w'), cls=BatterySampleEncoder)

In [11]:
[s.battery_id for s in samples]

[20,
 30,
 41,
 55,
 71,
 73,
 76,
 132,
 184,
 241,
 257,
 260,
 270,
 275,
 305,
 340,
 377,
 406,
 441,
 485,
 486,
 520,
 572,
 605,
 614,
 619,
 632,
 658,
 810,
 825,
 829,
 836,
 845,
 877,
 888,
 892,
 902,
 962,
 965,
 996]

Reload

In [12]:
data = json.load(open('available_samples.json', 'r'))
new_samples = [BatterySample.parse_obj(dic) for dic in data]

In [13]:
s = new_samples[0]
s

BatterySample(manufacturer='Conrad energy', composition=BatteryComposition(description='Li-based', cathode=None, anode=None, electrolyte=None), form_factor='2032', capacity=BatteryCapacity(nominal=45.0, actual=None, units='mAh'), battery_id=20, metadata=BatteryMetadata(name='Otis_20', creation_datetime=datetime.datetime(2020, 10, 8, 2, 49, 58, tzinfo=datetime.timezone.utc), creation_process='Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed mollis arcu ac risus vestibulum, ac commodo nunc euismod. Vivamus suscipit eros nec justo faucibus, et eleifend diam eleifend.'))